# Accelerating XGBoost on GPU Clusters with Dask

Our examples focus on the HIGGS dataset, a moderately sized classification problem from the UCI Machine Learning repository.  In the following sections, we start from basic data loading and preprocessing with GPU-accelerated Dask and Dask-ml. Then, train an XGBoost model on returned data with different configurations. Also, share some new features along the way. After that, we showcase how to compute SHAP value on a GPU cluster and the speedup we can obtain. Lastly, we share some optimization techniques with inference.

The following examples need to be run on a machine with at least one NVIDIA GPU, which can be a laptop or a cloud instance. One of the advantages of Dask is its flexibility that users can test their code on a laptop. They can also scale up the computation to clusters with a minimum amount of code changes.  Also, to set up the environment we need xgboost==1.4, dask, dask-ml, dask-cuda, and dask-cudf python packages, available from RAPIDS conda channels:

In [0]:
import os
from time import time
from typing import Tuple

import pandas as pd
import numpy as np
import cupy
import cudf
import dask
import dask_cudf
import dask_databricks
import dask_deltatable as ddt
import xgboost as xgb
from xgboost import dask as dxgb
from dask_ml.model_selection import train_test_split
from dask import dataframe as dd  #

In [0]:
client = dask_databricks.get_client()
client

Client 
 Client-eb001585-89a4-11ee-9783-ce42c69d1213 
 

 
 
 Connection method: Cluster object 
 Cluster type: dask_databricks.DatabricksCluster 
 
 

 
 
 
 Dashboard: https://dbc-dp-8721196619973675.cloud.databricks.com/driver-proxy/o/8721196619973675/1031-230718-l2ubf858/8087/status 
 
 
 
 

 

 

 
 
 Cluster Info 
 
 
 
 
 DatabricksCluster 
 1031-230718-l2ubf858 
 
 
 
 Dashboard: https://dbc-dp-8721196619973675.cloud.databricks.com/driver-proxy/o/8721196619973675/1031-230718-l2ubf858/8087/status 
 
 
 Workers: 2
 
 
 
 
 Total threads: 2
 
 
 Total memory: 30.65 GiB
 
 
 
 

 
 
 Scheduler Info 
 

 
 
 
 
 Scheduler 
 Scheduler-97faf66f-82df-4952-a352-349b7b4f9bbf 
 
 
 
 Comm: tcp://10.59.135.130:8786
 
 
 Workers: 2
 
 
 
 
 Dashboard: http://10.59.135.130:8087/status 
 
 
 Total threads: 2
 
 
 
 
 Started: 2 hours ago
 
 
 Total memory: 30.65 GiB
 
 
 
 
 

 
 
 Workers 
 

 
 
 
 
 
 
 Worker: tcp://10.59.145.57:36327 
 
 
 
 
 Comm: tcp://10.59.145.57:36327
 
 
 Total threads: 1
 
 
 
 
 Dashboard: http://10.59.145.57:42937/status 
 
 
 Memory: 15.33 GiB
 
 
 
 
 Nanny: tcp://10.59.145.57:33155
 
 
 
 
 
 Local directory: /tmp/dask-scratch-space/worker-9b10jylt
 
 

 
 
 
 GPU: Tesla T4
 
 
 GPU memory: 15.00 GiB
 
 
 

 

 
 
 
 
 
 
 
 
 
 
 Worker: tcp://10.59.158.38:45163 
 
 
 
 
 Comm: tcp://10.59.158.38:45163
 
 
 Total threads: 1
 
 
 
 
 Dashboard: http://10.59.158.38:44757/status 
 
 
 Memory: 15.33 GiB
 
 
 
 
 Nanny: tcp://10.59.158.38:33655
 
 
 
 
 
 Local directory: /tmp/dask-scratch-space/worker-c0jj65qr
 
 

 
 
 
 GPU: Tesla T4
 
 
 GPU memory: 15.00 GiB

## Load dataset
Given a cluster, we start loading the data into GPUs.  Because the data is loaded multiple times during parameter tuning, we convert the CSV file into Parquet format for better performance.  This can be easily done using dask_cudf:


In [0]:
# CONVERT TO DELTA parquet.`s3://my-bucket/parquet-data`;

In [0]:
# The next three lines should be run only ONCE!
data = spark.read.load("dbfs:/dbfs/databricks/skirui/part_10.parquet", format="parquet")
data

Out[4]: DataFrame[__null_dask_index__: bigint, label: float, feature-01: float, feature-02: float, feature-03: float, feature-04: float, feature-05: float, feature-06: float, feature-07: float, feature-08: float, feature-09: float, feature-10: float, feature-11: float, feature-12: float, feature-13: float, feature-14: float, feature-15: float, feature-16: float, feature-17: float, feature-18: float, feature-19: float, feature-20: float, feature-21: float, feature-22: float, feature-23: float, feature-24: float, feature-25: float, feature-26: float, feature-27: float, feature-28: float]

In [0]:
# Write to Delta table using the schema inferred from `data`
table_name = "part10_higgs"
data.write.saveAsTable(table_name)

In [0]:
display(spark.sql("DESCRIBE DETAIL part10_higgs"))

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,0046d036-9050-400a-b666-1e5c0e52ef1d,spark_catalog.default.part10_higgs,null,dbfs:/user/hive/warehouse/part10_higgs,2023-11-23T02:06:55.082+0000,2023-11-23T02:07:03.000+0000,List(),1,25733030,Map(),1,2,"List(appendOnly, invariants)",Map()


In [0]:
dd = ddt.read_deltalake("/dbfs/user/hive/warehouse/part10_higgs/")
dd.head()

,__null_dask_index__,label,feature-01,feature-02,feature-03,feature-04,feature-05,feature-06,feature-07,feature-08,...,feature-19,feature-20,feature-21,feature-22,feature-23,feature-24,feature-25,feature-26,feature-27,feature-28
0,0,1.0,1.485302,0.400247,-0.812006,0.597350,1.529408,1.183657,-0.509004,1.387762,...,-0.341085,0.160763,0.000000,0.855138,1.011769,1.119636,0.960537,0.985170,0.939820,0.802873
1,1,0.0,0.736795,-1.607103,0.777777,1.905104,0.246345,2.179335,-0.665460,-1.511726,...,-1.029830,0.283399,3.101961,0.925825,1.065189,0.983639,0.887213,0.668263,0.866008,0.854103
2,2,1.0,0.731304,0.661271,0.273963,2.320010,-1.260483,0.779028,-0.560495,-1.423580,...,-0.351912,-0.036888,0.000000,0.898584,1.090466,1.772436,1.279596,0.747488,0.986794,0.958956
3,3,1.0,0.556531,0.424596,0.289499,0.984535,1.475206,1.219933,0.342594,0.779613,...,-0.681710,1.034749,3.101961,0.937095,1.056379,0.989476,0.613769,0.870377,0.767229,0.673387
4,4,1.0,0.486987,0.859960,-1.650956,0.724159,0.275873,0.919278,1.167456,-0.910783,...,1.630207,-0.658947,0.000000,2.282790,1.454625,0.988534,0.982069,1.391338,1.130450,0.947944


In [0]:
ddf = dask_cudf.from_dask_dataframe(dd)
ddf

,__null_dask_index__,label,feature-01,feature-02,feature-03,feature-04,feature-05,feature-06,feature-07,feature-08,feature-09,feature-10,feature-11,feature-12,feature-13,feature-14,feature-15,feature-16,feature-17,feature-18,feature-19,feature-20,feature-21,feature-22,feature-23,feature-24,feature-25,feature-26,feature-27,feature-28
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [0]:
type(ddf)

Out[9]: dask_cudf.core.DataFrame

In [0]:
# def read_deltatable():
#   # The next three lines should be run only ONCE!
#   data = spark.read.load("dbfs:/dbfs/databricks/skirui/part_10.parquet",
#                           format="parquet")

#   # Write to Delta table using the schema inferred from `data`
#   table_name = 'part10_higgs'
#   data.write.saveAsTable(table_name)
#   display(spark.sql('DESCRIBE DETAIL part10_higgs'))

#   # read from delta table using dask
#   dd = ddt.read_deltalake("/dbfs/user/hive/warehouse/part10_higgs/")

#   # dask.config.set({"dataframe.backend": "cudf"})
#   ddf = dask_cudf.from_dask_dataframe(dd)
#   return ddf

After data loading, we prepare the training/validation splits:

In [0]:
from distributed import wait


def load_higgs(
    ddf,
) -> Tuple[
    dask_cudf.core.DataFrame,
    dask_cudf.core.Series,
    dask_cudf.core.DataFrame,
    dask_cudf.core.Series,
]:
    df = ddf.copy()
    y = df["label"]
    X = df[df.columns.difference(["label"])]

    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.33, random_state=42
    )
    X_train, X_valid, y_train, y_valid = client.persist(
        [X_train, X_valid, y_train, y_valid]
    )
    wait([X_train, X_valid, y_train, y_valid])

    return X_train, X_valid, y_train, y_valid

In [0]:
X_train, X_valid, y_train, y_valid = load_higgs(ddf)

/databricks/python/lib/python3.10/site-packages/dask_ml/model_selection/_split.py:462: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


In [0]:
print(f"len(X_train): {len(X_train)}")
print(f"len(X_valid): {len(X_valid)}")
print(f"len(y_train): {len(y_train)}")
print(f"len(y_valid): {len(y_valid)}")

len(X_train): 245918
len(X_valid): 121549
len(y_train): 245918
len(y_valid): 121549


In the preceding example, we use dask-cudf for loading data from the disk, and the train_test_split function from dask-ml for splitting up the dataset.  Most of the time, the GPU backend of dask works seamlessly with utilities in dask-ml and we can accelerate the entire ML pipeline.

## Training with early stopping
One of the most frequently requested features is early stopping support for the Dask interface.  In the XGBoost 1.4 release, not only can we specify the number of stopping rounds, but also develop customized early stopping strategies.  For the simplest case, providing stopping rounds to the train function enables early stopping:

There are two things to notice here.  Firstly, we specify the number of rounds to trigger early stopping for training.  XGBoost will stop the training process once the validation metric fails to improve in consecutive X rounds, where X is the number of rounds specified for early stopping.  Secondly, we use a data type called DaskDeviceQuantileDMatrix for training but DaskDMatrix for validation.  DaskDeviceQuantileDMatrix is a drop-in replacement of DaskDMatrix for GPU-based training inputs that avoids extra data copies.

In [0]:
!pip list

Package                      Version
---------------------------- -------------
absl-py                      1.4.0
asttokens                    2.2.1
astunparse                   1.6.3
backcall                     0.2.0
blinker                      1.4
bokeh                        3.2.2
cachetools                   5.3.1
certifi                      2023.7.22
charset-normalizer           3.2.0
click                        8.1.7
cloudpickle                  3.0.0
comm                         0.1.3
contourpy                    1.1.0
cryptography                 3.4.8
cubinlinker-cu11             0.3.0.post1
cuda-python                  11.8.3
cudf-cu11                    23.10.2
cuml-cu11                    23.10.0
cupy-cuda11x                 12.2.0
cycler                       0.11.0
dask                         2023.9.2
dask-cuda                    23.10.0
dask-cudf-cu11               23.10.2
dask-databricks              0.3.0
dask-deltatable              0.3.1
dask-glm               

In [0]:
def fit_model_es(client, X, y, X_valid, y_valid) -> dxgb.Booster:
    early_stopping_rounds = 5
    Xy = dxgb.DaskDeviceQuantileDMatrix(client, X, y)
    Xy_valid = dxgb.DaskDMatrix(client, X_valid, y_valid)
    # train the model
    booster = dxgb.train(
        client,
        {
            "objective": "binary:logistic",
            "eval_metric": "error",
            "tree_method": "gpu_hist",
        },
        Xy,
        evals=[(Xy_valid, "Valid")],
        num_boost_round=1000,
        early_stopping_rounds=early_stopping_rounds,
    )["booster"]
    return booster

In [0]:
booster = fit_model_es(client, X=X_train, y=y_train, X_valid=X_valid, y_valid=y_valid)
booster

/databricks/python/lib/python3.10/site-packages/xgboost/dask.py:704: FutureWarning: Please use `DaskQuantileDMatrix` instead.
  warnings.warn("Please use `DaskQuantileDMatrix` instead.", FutureWarning)


---------------------------------------------------------------------------
XGBoostError                              Traceback (most recent call last)
File <command-241619327411160>:1
----> 1 booster = fit_model_es(client, X=X_train, y=y_train, X_valid=X_valid, y_valid=y_valid) 
      2 booster

File <command-241619327411108>:6, in fit_model_es(client, X, y, X_valid, y_valid)
      4 Xy_valid = dxgb.DaskDMatrix(client, X_valid, y_valid)
      5 # train the model
----> 6 booster = dxgb.train(
      7     client,
      8     {
      9         "objective": "binary:logistic",
     10         "eval_metric": "error",
     11         "tree_method": "gpu_hist",
     12     },
     13     Xy,
     14     evals=[(Xy_valid, "Valid")],
     15     num_boost_round=1000,
     16     early_stopping_rounds=early_stopping_rounds,
     17 )["booster"]
     18 return booster

File /databricks/python/lib/python3.10/site-packages/xgboost/core.py:729, in require_keyword_args.<locals>.throw_if.<locals>.inne

## Customized objective and evaluation metric

XGBoost is designed to be scalable through customized objective functions and metrics. In 1.4, this feature is brought to the dask interface. The requirement is exactly the same as for the single node interface:

Optional: In the example below we use the custom objective function and metric to implement a logistic regression model along with early stopping. Note that the function returns both gradient and hessian, which XGBoost uses to optimize the model.  Also, the parameter named metric_name needs to be specified in our callback. It is used to inform XGBoost that the custom error function should be used for evaluating early stopping criteri

In [0]:
# def fit_model_customized_objective(client, X, y, X_valid, y_valid) -> dxgb.Booster:
#     def logit(predt: np.ndarray, Xy: xgb.DMatrix) -> Tuple[np.ndarray, np.ndarray]:
#         predt = 1.0 / (1.0 + np.exp(-predt))
#         labels = Xy.get_label()
#         grad = predt - labels
#         hess = predt * (1.0 - predt)
#         return grad, hess

#     def error(predt: np.ndarray, Xy: xgb.DMatrix) -> Tuple[str, float]:
#         label = Xy.get_label()
#         r = np.zeros(predt.shape)
#         predt = 1.0 / (1.0 + np.exp(-predt))
#         gt = predt > 0.5
#         r[gt] = 1 - label[gt]
#         le = predt <= 0.5
#         r[le] = label[le]
#         return "CustomErr", float(np.average(r))

#     # Use early stopping with custom objective and metric.
#     early_stopping_rounds = 5
#     # Specify the metric we want to use for early stopping.
#     es = xgb.callback.EarlyStopping(
#     rounds=early_stopping_rounds, save_best=True, metric_name="CustomErr"
#     )

#     Xy = dxgb.DaskDeviceQuantileDMatrix(client, X, y)
#     Xy_valid = dxgb.DaskDMatrix(client, X_valid, y_valid)
#     booster = dxgb.train(
#         client,
#         {"eval_metric": "error", "tree_method": "gpu_hist"},
#         Xy,
#         evals=[(Xy_valid, "Valid")],
#         num_boost_round=1000,
#         obj=logit,  # pass the custom objective
#         feval=error,  # pass the custom metric
#         callbacks=[es],
#     )["booster"]
#    return booster

## Explaining the model
After obtaining our first model, we might want to explain predictions using SHAP.  SHAP(SHapley Additive exPlanations) is a game theoretic approach to explain the output of machine learning models based on Shapley Value.  For details about the algorithm, please refer to the papers.  As XGBoost now has support for GPU-accelerated Shapley values, we extend this feature to the Dask interface. Now, users can compute shap values on distributed GPU clusters. This is enabled by the significantly improved predict function and the GPUTreeShap library:

In [0]:
def explain(client, model, X):
    # Use array instead of dataframe in case of output dim is greater than 2.
    X_array = X.values
    contribs = dxgb.inplace_predict(
        client, model, X_array, pred_contribs=True, validate_features=False
    )
    # Use the result for further analysis
    return contribs

In [0]:
contribs = explain(client, model=booster, X=X_train)